In [4]:
import torch
from torch.nn.utils import prune
from transformers import BertTokenizer, BertForSequenceClassification



device = "mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else "cpu"
tensor = torch.FloatTensor([1, 2, 3]).to(device)

tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased",
    do_lower_case=False,
)
model = BertForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased",
    num_labels=2
)

torch.save(model.state_dict(), "../models/BertForSequenceClassification.pt")

model.load_state_dict(torch.load("../models/BertForSequenceClassification.pt"))

print("가지치기 적용 전:")
print(model.bert.encoder.layer[0].attention.self.key.weight)

parameters = [
    (model.bert.embeddings.word_embeddings, "weight"),
    (model.bert.encoder.layer[0].attention.self.key, "weight"),
    (model.bert.encoder.layer[1].attention.self.key, "weight"),
    (model.bert.encoder.layer[2].attention.self.key, "weight"),
]
prune.global_unstructured(
    parameters=parameters,
    pruning_method=prune.L1Unstructured,
    amount=0.2
)

print("가지치기 적용 후:")
print(model.bert.encoder.layer[0].attention.self.key.weight)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


가지치기 적용 전:
Parameter containing:
tensor([[-0.0259, -0.0599, -0.0395,  ...,  0.0083, -0.0271, -0.0043],
        [ 0.0419,  0.0561,  0.0044,  ..., -0.0042,  0.0200,  0.0163],
        [ 0.0160, -0.0439, -0.0092,  ..., -0.0115, -0.0879, -0.0514],
        ...,
        [-0.0310, -0.0218,  0.0345,  ..., -0.0010,  0.0107,  0.0088],
        [-0.0756, -0.0755, -0.0067,  ..., -0.0132, -0.0330, -0.0986],
        [ 0.0159, -0.0423, -0.0061,  ...,  0.0445, -0.0184,  0.0410]],
       requires_grad=True)
가지치기 적용 후:
tensor([[-0.0259, -0.0599, -0.0395,  ...,  0.0000, -0.0271, -0.0000],
        [ 0.0419,  0.0561,  0.0000,  ..., -0.0000,  0.0200,  0.0163],
        [ 0.0160, -0.0439, -0.0000,  ..., -0.0115, -0.0879, -0.0514],
        ...,
        [-0.0310, -0.0218,  0.0345,  ..., -0.0000,  0.0000,  0.0000],
        [-0.0756, -0.0755, -0.0000,  ..., -0.0132, -0.0330, -0.0986],
        [ 0.0159, -0.0423, -0.0000,  ...,  0.0445, -0.0184,  0.0410]],
       grad_fn=<MulBackward0>)


In [5]:
import os

file_path = "../models/BertForSequenceClassification.pt"
print("File exists:", os.path.exists(file_path))
print("Absolute path:", os.path.abspath(file_path))


File exists: True
Absolute path: /Users/chohi/project/ai/DeepLearningDeepen/models/BertForSequenceClassification.pt
